In [1]:
pip install rank_bm25

In [2]:
from rank_bm25 import BM25Okapi

In [3]:
corpus = [
    "The cat, commonly referred to as the domestic cat or house cat, is a small domesticated carnivorous mammal.",
    "The dog is a domesticated descendant of the wolf.",
    "Humans are the most common and widespread species of primate, and the last surviving species of the genus Homo.",
    "The scientific name Felis catus was proposed by Carl Linnaeus in 1758"
]

In [4]:
tokenized_corpus = [doc.split(" ") for doc in corpus]

In [5]:
tokenized_corpus

[['The',
  'cat,',
  'commonly',
  'referred',
  'to',
  'as',
  'the',
  'domestic',
  'cat',
  'or',
  'house',
  'cat,',
  'is',
  'a',
  'small',
  'domesticated',
  'carnivorous',
  'mammal.'],
 ['The', 'dog', 'is', 'a', 'domesticated', 'descendant', 'of', 'the', 'wolf.'],
 ['Humans',
  'are',
  'the',
  'most',
  'common',
  'and',
  'widespread',
  'species',
  'of',
  'primate,',
  'and',
  'the',
  'last',
  'surviving',
  'species',
  'of',
  'the',
  'genus',
  'Homo.'],
 ['The',
  'scientific',
  'name',
  'Felis',
  'catus',
  'was',
  'proposed',
  'by',
  'Carl',
  'Linnaeus',
  'in',
  '1758']]

In [6]:
bm25 = BM25Okapi(tokenized_corpus)

In [7]:
query = "The cat"
tokenized_query = query.split(" ")

In [8]:
tokenized_query

['The', 'cat']

In [9]:
doc_scores = bm25.get_scores(tokenized_query)

# scores for documents 1, 2, 3, and 4
print(doc_scores)

[0.92061135 0.20898199 0.         0.18788848]


In [10]:
#We can try with another query
query_2 = "The dog"
tokenized_query_2 = query_2.split(" ")

tokenized_query_2

['The', 'dog']

In [11]:
doc_scores = bm25.get_scores(tokenized_query_2)

# scores for documents 1, 2, 3, and 4
print(doc_scores)

[0.15633023 1.23067172 0.         0.18788848]


#Semantic Similiarity Search

In [12]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 6.5 MB/s eta 0:00:00


In [13]:
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [14]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
# The documents to encode
corpus = [
    "The cat, commonly referred to as the domestic cat or house cat, is a small domesticated carnivorous mammal.",
    "The dog is a domesticated descendant of the wolf.",
    "Humans are the most common and widespread species of primate, and the last surviving species of the genus Homo.",
    "The scientific name Felis catus was proposed by Carl Linnaeus in 1758"
]

In [16]:
# Calculate embeddings by calling model.encode()
document_embeddings = model.encode(corpus)

# Sanity check
print(document_embeddings.shape)

(4, 384)


In [17]:
query = "The cat"
query_embedding = model.encode(query)

In [18]:
from sentence_transformers.util import cos_sim

# Compute cosine_similarity between documents and query
scores = cos_sim(document_embeddings, query_embedding)
scores

tensor([[0.5716],
        [0.2904],
        [0.0942],
        [0.3157]])

In [19]:
query_embedding = model.encode("feline")

scores = cos_sim(document_embeddings, query_embedding)

print(scores)

tensor([[0.4007],
        [0.3837],
        [0.0966],
        [0.3804]])


#Hybrid Search
Combining Keyword and Semantic Searching
We can combine the results of BM25 and cosine similarity using Reciprocal Rank Fusion (RRF). RRF is a simple algorithm for combining the rankings of different scoring functions

In [20]:
corpus = [
    "The cat, commonly referred to as the domestic cat or house cat, is a small domesticated carnivorous mammal.",
    "The dog is a domesticated descendant of the wolf.",
    "Humans are the most common and widespread species of primate, and the last surviving species of the genus Homo.",
    "The scientific name Felis catus was proposed by Carl Linnaeus in 1758",
]
query = "The cat"

bm25_ranking = [1, 2, 4, 3] # scores = [0.92932018 0.21121974 0. 0.1901173]
cosine_ranking = [1, 3, 4, 2] # scores = [0.5716, 0.2904, 0.0942, 0.3157]

In [21]:
import numpy as np

def scores_to_ranking(scores: list[float]) -> list[int]:
    """Convert float scores into int rankings (rank 1 is the best)"""
    return np.argsort(scores)[::-1] + 1


def rrf(keyword_rank: int, semantic_rank: int) -> float:
    """Combine keyword rank and semantic rank into a hybrid score."""
    k = 60
    rrf_score = 1 / (k + keyword_rank) + 1 / (k + semantic_rank)
    return rrf_score

In [23]:
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

In [24]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [25]:
def hybrid_search(
    query: str, corpus: list[str], encoder_model: SentenceTransformer
) -> list[int]:
    # bm25
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    tokenized_query = query.split(" ")
    print(f"tokenized query: {tokenized_query}")
    bm25 = BM25Okapi(tokenized_corpus)
    bm25_scores = bm25.get_scores(tokenized_query)
    print(f"bm25 scores: {bm25_scores}")
    bm25_ranking = scores_to_ranking(bm25_scores)
    print(f"bm25 ranking: {bm25_ranking}")
    # embeddings
    document_embeddings = model.encode(corpus)
    query_embedding = model.encode(query)
    cos_sim_scores = cos_sim(document_embeddings, query_embedding).flatten().tolist()
    print(f"cosine sim scores: {cos_sim_scores}")
    cos_sim_ranking = scores_to_ranking(cos_sim_scores)
    print(f"cosine sim ranking: {cos_sim_ranking}")

        # combine rankings into RRF scores
    hybrid_scores = []
    for i, doc in enumerate(corpus):
        document_ranking = rrf(bm25_ranking[i], cos_sim_ranking[i])
        print(f"Document {i} has the rrf score {document_ranking}")
        hybrid_scores.append(document_ranking)

    # convert RRF scores into final rankings
    hybrid_ranking = scores_to_ranking(hybrid_scores)
    return hybrid_ranking

In [26]:
hybrid_ranking = hybrid_search(
    query="What is the scientifc name for cats?", corpus=corpus, encoder_model=model
)
print(hybrid_ranking)

tokenized query: ['What', 'is', 'the', 'scientifc', 'name', 'for', 'cats?']
bm25 scores: [0.15633023 0.20898199 0.26805423 0.9185659 ]
bm25 ranking: [4 3 2 1]
cosine sim scores: [0.5001033544540405, 0.3556438982486725, 0.17555519938468933, 0.5748379826545715]
cosine sim ranking: [4 1 2 3]
Document 0 has the rrf score 0.03125
Document 1 has the rrf score 0.032266458495966696
Document 2 has the rrf score 0.03225806451612903
Document 3 has the rrf score 0.032266458495966696
[4 2 3 1]
